Now we will continue on the [Conversation AI](https://conversationai.github.io/) dataset seen in [week 4 homework and lab](https://github.com/MIDS-scaling-up/v2/tree/master/week04). 
We shall use a version of pytorch BERT for classifying comments found at [https://github.com/huggingface/pytorch-pretrained-BERT](https://github.com/huggingface/pytorch-pretrained-BERT).  

The original implementation of BERT is optimised for TPU. Google released some amazing performance improvements on TPU over GPU, for example, see [here](https://medium.com/@ranko.mosic/googles-bert-nlp-5b2bb1236d78) - *BERT relies on massive compute for pre-training ( 4 days on 4 to 16 Cloud TPUs; pre-training on 8 GPUs would take 40–70 days).*. In response, Nvidia released [apex](https://devblogs.nvidia.com/apex-pytorch-easy-mixed-precision-training/), which gave mixed precision training. Weights are stored in float32 format, but calculations, like forward and backward propagation happen in float16 - this allows these calculations to be made with a [4X speed up](https://github.com/huggingface/pytorch-pretrained-BERT/issues/149).  

We shall apply BERT to the problem for classifiying toxicity, using apex from Nvidia. We shall compare the impact of hardware by running the model on a V100 and P100 and comparing the speed and accuracy in both cases.   

This script relies heavily on an existing [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967). 
  
*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

In [1]:
import sys, os
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

In [2]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

Change the PATH variable to whereever your `week06/hw` directory is located.  
**For the final run we would like you to have a train_size of at least 1 Million rows, and a valid size of at least 500K rows. When you first run the script, feel free to work with a reduced train and valid size for speed.** 

In [52]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 1
PATH = '/root/v2/week06/hw' # /root/v2/week06/hw"
DATA_DIR = os.path.join(PATH, "data")
WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
train_size= 1000000 # 1000000 
valid_size= 500000  # 500000

This should be the files you downloaded earlier when you ran `download.sh`

In [4]:
os.listdir(DATA_DIR)

['download.sh',
 'uncased_L-12_H-768_A-12',
 'test.csv',
 'cased_L-12_H-768_A-12',
 'train.csv']

We shall install pytorch BERT implementation.   
If you would like to experiment with or view any code (purely optional, and not graded :) ), you can copy the files from the repo https://github.com/huggingface/pytorch-pretrained-BERT  

In [5]:
%%capture
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import BertConfig

We shall now load the model. When you run this, comment out the `capture` command to understand the archecture.

In [6]:
#%%capture
# Translate model from tensorflow to pytorch
BERT_MODEL_PATH = os.path.join(DATA_DIR, 'uncased_L-12_H-768_A-12')
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
                            os.path.join(BERT_MODEL_PATH, 'bert_model.ckpt'),
                            os.path.join(BERT_MODEL_PATH, 'bert_config.json'), 
                            os.path.join(WORK_DIR, 'pytorch_model.bin'))

shutil.copyfile(os.path.join(BERT_MODEL_PATH, 'bert_config.json'), \
                os.path.join(WORK_DIR, 'bert_config.json'))
# This is the Bert configuration file
bert_config = BertConfig(os.path.join(WORK_DIR, 'bert_config.json'))

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /root/v2/week06/hw/data/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768]
Loadi

'/root/v2/week06/hw/workingdir/bert_config.json'

Bert needs a special formatting of sentences, so we have a sentence start and end token, as well as separators.   
Thanks to this [script](https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming) for a fast convertor of the sentences.

In [53]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

Now we load the BERT tokenizer and convert the sentences.

In [54]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_all = pd.read_csv(os.path.join(DATA_DIR, "train.csv")).sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['comment_text'] = train_all['comment_text'].astype(str) 

sequences = convert_lines(train_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500000 records



33724
CPU times: user 34min 4s, sys: 19.3 s, total: 34min 24s
Wall time: 34min 6s


Let us look at how the tokenising works in BERT, see below how it recongizes misspellings - words the model never saw. 

In [55]:
train_all[["comment_text", 'target']].head()

,comment_text,target
458232,It's difficult for many old people to keep up ...,0.000000
272766,She recognized that her tiny-handed husband is...,0.166667
339129,"HPHY76,\nGood for you for thinking out loud, w...",0.000000
773565,And I bet that in the day you expected your Je...,0.500000
476233,Kennedy will add a much needed and scientifica...,0.000000


Lets tokenize some text (I intentionally mispelled some words to check berts subword information handling)

In [11]:
text = 'Hi, I am learning new things in w251 about deep learning the cloud and teh edge.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i am learning new things in w ##25 ##1 about deep learning the cloud and te ##h edge .'

Added start and end token and convert to ids. This is how it is fed into BERT.

In [12]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2572 4083 2047 2477 1999 1059 17788 2487 2055 2784 4083 1996 6112 1998 8915 2232 3341 1012 102'

When BERT converts this sentence to a torch tensor below is shape of the stored tensors.  
We have 12 input tensors, while the sentence tokens has length 23; where are can you see the 23 tokens in the tensors ?... **Feel free to post in slack or discuss in class**

In [15]:
# put input on gpu and make prediction
bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids]).cuda())

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

Sentence tokens ['[CLS]', 'hi', ',', 'i', 'am', 'learning', 'new', 'things', 'in', 'w', '##25', '##1', 'about', 'deep', 'learning', 'the', 'cloud', 'and', 'te', '##h', 'edge', '.', '[SEP]']
Number of tokens 23
Tensor shapes : [(1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768), (1, 23, 768)]
Number of torch tensors : 12


In [16]:
print('Tensor shapes : {}'.format([b.cpu().detach().numpy()[0] for b in bert_output[0]]))

Tensor shapes : [array([[ 0.12506592,  0.23669545, -0.16170123, ...,  0.08339483,
         0.01473224,  0.0118218 ],
       [ 0.6461339 , -0.58027226,  0.7776804 , ..., -0.3316052 ,
        -0.5405882 , -1.023122  ],
       [-0.4219209 ,  0.06394117, -0.34287015, ..., -0.37609768,
         0.6959227 ,  0.08924114],
       ...,
       [ 0.5054876 , -1.051371  , -0.18810612, ..., -0.4343829 ,
         1.2711586 , -0.06678137],
       [-0.2102965 ,  0.3366063 , -0.24493271, ...,  0.11911466,
         0.42725167,  0.20310833],
       [ 0.00525613,  0.14335708, -0.09598022, ..., -0.17090887,
         0.31359437, -0.16866437]], dtype=float32), array([[-0.07822789, -0.04128889, -0.5235031 , ...,  0.25773108,
         0.07842778, -0.15379667],
       [ 1.3754444 , -0.13587219,  1.4553052 , ...,  0.01933745,
        -0.18513441, -1.4833139 ],
       [-0.5493904 ,  0.06952696, -0.2504364 , ..., -0.33933067,
         0.2238872 , -0.15779531],
       ...,
       [ 0.7297506 , -1.8254522 , -0.26906

As it is a binary problem, we change our target to [0,1], instead of float.   
We also split the dataset into a training and validation set, 

In [56]:
train_all['target']=(train_all['target']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['target']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['target']].values[train_size:]

In [57]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

**From here on in we would like you to run BERT.**   
**Please do rely on the script available -  [Kaggle kernel](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) from [yuval r](https://www.kaggle.com/yuval6967) - for at least the first few steps up to training and prediction.**


**1)**   
**Load the training set to a training dataset. For this you need to load the X sequences and y objects to torch tensors**   
**You can use `torch.utils.data.TensorDataset` to input these into a train_dataset.**

In [58]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(X,dtype=torch.long),
    torch.tensor(y,dtype=torch.float))

**2)**  
**Set your learning rate and batch size; and optionally random seeds if you want reproducable results**   
**Load your pretrained BERT using `BertForSequenceClassification`**   
**Initialise the gradients and place the model on cuda, set up your optimiser and decay parameters**
**Initialise the model with `apex` (we imprted this as `amp`) for mixed precision training**

In [59]:
# Start with very small learning rate for BERT finetuning (typically between 2e-5 and 5e-5)
lr=2e-5

# Set batch size - increments of 32 work nicely with GPU - watch out for memory limitations
batch_size = 32

accumulation_steps=2

# Random Seed to Use - will make the results reproducible
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Initialize the Bert model with the pretrained weights
model = BertForSequenceClassification.from_pretrained(WORK_DIR,cache_dir=None,num_labels=1)

# Zero out the gradients. We are finetuning and don't want the old gradient information
model.zero_grad()

# Port the model to the GPU
model = model.to(device)

# Get a list of model parameters
param_optimizer = list(model.named_parameters())

# No weight decay on bias terms
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

# Loop over model parameters set the weight decay terms (used in the GD optimizer)
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

train = train_dataset

# Training Optimization - update only every other batch
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)


**3)**  
**Start training your model by iterating through batches in a single epoch of the data**

In [60]:
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()   # Bug fix - thanks to @chinhuic
    for i,(x_batch, y_batch) in tk0:
#        optimizer.zero_grad()
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)




**4)**  
**Store your trained model to disk, you will need it if you choose section 8C.**

In [62]:
output_model_file = "bert_pytorch_p100.bin"
torch.save(model.state_dict(), output_model_file)

**5)**   
**Now make a prediction for your validation set.**  

In [63]:
# Run validation
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

**6)**  
**In the yuval's kernel he get a metric based on the metric for the jigsaw competition - it is quite complicated. Instead, we would like you to measure the `AUC`, similar to how you did in homework 04. You can compare the results to HW04**  
*A tip, if your score is lower than homework 04 something is wrong....*

## Score from subset of data
### 0.79870

## Score after training on 1 million rows
### 0.9699

In [ ]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_val, valid_preds)))

**7)**  
**Can you show/print the validation sentences predicted with the highest and lowest toxicity ?**

Getting the sentence with max toxicity is a matter of finding the prediction with the highest prediction probability. We can simply use an argmax as the predictions are just an array. With this argmax, we have the array index (relative to the validation set). We can index into the validation sentences and then decode the ids back into tokens. But then we still have masked words as shown below. The easier thing is to go back to our source data, and index into the dataframe after offsetting for the training data set size. The cells below show that both methods result in the same answer.

In [51]:
np.max(valid_preds), np.min(valid_preds)

(-1.6416015625, -2.91015625)

In [42]:
X_val[np.argmax(valid_preds)]

array([  101,  8951,  2231, 18574,  2085,   999,   999,   999,   999,
        2205,  9541,  9541,  9541,  6057,   999,   999,   999,   999,
        1996,  3649,  2017,  2024,  2008,  4122,  2000,  6033,  2256,
        2231,  1998,  2942,  2554,  2036, 17248,  2231,  3860,   999,
         999,  1998,  2066,  1996, 10276,  1010,  2182,  2017,  2024,
        2153,   999,  2469,  1011,  2074,  3046,  2000,  1999,  6767,
        3489,  2231,  3860,  1012,  2054,  1037, 20342,  5081,  1012,
        2292,  1005,  1055,  2156,  2017,  3046,  2007,  2035,  2115,
       23653,  1044, 22571, 10085,  6935,  2100,  1012,  2054,  1037,
       10424,  6799,  2378,  5390,  3676,  3762, 20716,  2017,  2024,
        2667,  2000,  1999, 25608,  2063,  2115, 11808, 14021,  1011,
        1056,  1999,  2256, 11450,  1998,  2017,  2215,  2256,  2231,
        3860,  1029,  1045,  2572,  2019,  2137,  6926, 28428,  2026,
        3083,  7450,  2916,  1012,  2178,  6707,  1999,  2137,  1010,
       27661,  1012,

In [48]:
' '.join(tokenizer.convert_ids_to_tokens(X_val[np.argmax(valid_preds)][1:]))


"citing government statutes now ! ! ! ! too ##oo ##oo ##oo funny ! ! ! ! the whatever you are that wants to destroy our government and civil society also cites government protection ! ! and like the wizard , here you are again ! sure - just try to in ##vo ##ke government protection . what a lame threat . let ' s see you try with all your unbelievable h ##yp ##oc ##ris ##y . what a fr ##ick ##in cry ##ba ##by bully you are trying to in ##fus ##e your nasty sh - t in our conversations and you want our government protection ? i am an american citizen exercising my 1st amendment rights . another mistake in american , comrade . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [38]:
train_all['comment_text'].iloc[train_size+np.argmax(valid_preds)]

"Citing government statutes now!!!!  TOOOOOOOO Funny!!!!  The whatever you are that wants to destroy our government and civil society also cites government protection!!  And like the wizard, here you are again!  Sure -  just try to invoke government protection. What a lame threat.  Let's see you try with all your unbelievable hypocrisy.  What a frickin crybaby bully you are trying to infuse your nasty sh-t in our conversations AND you want our government protection?  I am an American citizen exercising my 1st amendment rights.  Another mistake in American, Comrade."

In [50]:
train_all['comment_text'].iloc[train_size+np.argmin(valid_preds)]

"We'll see..."

**8)**  
**Pick only one of the below items and complete it. The last two will take a good amount of time (and partial success on them is fine), so proceed with caution on your choice of items :)** 
  
  
**A. Can you train on two epochs ?**

**B. Can you change the learning rate and improve validation score ?**
   
**C. Make a prediction on the test data set with your downloaded model and submit to Kaggle to see where you score on public LB - check out [Abhishek's](https://www.kaggle.com/abhishek) script - https://www.kaggle.com/abhishek/pytorch-bert-inference**  
  
**D. Get BERT running on the tx2 for a sample of the data.** 
  
**E. Finally, and very challenging -- the `BertAdam` optimiser proved to be suboptimal for this task. There is a better optimiser for this dataset in this script [here](https://www.kaggle.com/cristinasierra/pretext-lstm-tuning-v3). Check out the `custom_loss` function. Can you implement it ? It means getting under the hood of the `BertForSequenceClassification` at the source repo and implementing a modified version locally .  `https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/modeling.py`**